# Libraries

In [ ]:
# Import necessary libraries

import pandas as pd
import numpy as np
import glob
import re
import cv2 as cv
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50, inception_v3, densenet
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model, layers
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

2023-08-09 12:54:46.430417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 12:54:47.844223: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alvin/anaconda3/envs/asr/lib/python3.9/site-packages/cv2/../../lib64:
2023-08-09 12:54:47.844327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alvin/anaconda3/envs/asr/lib/python3.9/site-pack

# Custom Metrics by Nabasa Rodrick

## F1 Score

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

# Instantiate precision and recall metric objects
p = Precision(thresholds=0)
r = Recall(thresholds=0)

def f1_score(y_true, y_pred):
  """
  Function that calculates and returns
  f1_score using Precision and Recall
  """

  def precision(y_true, y_pred):
    """
    Function that calculates and returns precision
    """
    p.update_state(y_true, y_pred)
    return p.result()

  def recall(y_true, y_pred):
    """
    Function that calculates and returns recall
    """
    r.update_state(y_true, y_pred)
    return r.result()

  precision = precision(y_true, y_pred)
  recall = recall(y_true, y_pred)

  result = tf.math.multiply_no_nan(
      tf.constant(2, tf.float32),
      tf.math.divide_no_nan(
          tf.math.multiply_no_nan(
              precision,
              recall
          ),
          tf.math.add(
              precision,
              recall
          )
      )
  )

  return result

# Augmentations

In [ ]:
def random_rotation(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

  degree = np.random.randint(-15,15,1)[0]

  rows,cols = img.shape
  rotation_matrix = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0), degree, 1)
  img = cv.warpAffine(img, rotation_matrix, (cols,rows))

  img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
def hist_eq(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

  contrast = np.random.uniform(1.0,2.0)
  clahe = cv.createCLAHE(clipLimit=contrast, tileGridSize=(16,16))
  img = clahe.apply(img)

  img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
# Smoothing to remove noise
def smoothing(img):
  img_shape = img.shape
  img = tf.squeeze(img)

  img = img.numpy().astype(np.uint8)

  img = cv.GaussianBlur(img,(3,3),0)

  img = tf.convert_to_tensor(img, dtype=tf.float32)

  img = tf.expand_dims(img, 0)
  img.set_shape(img_shape)

  return img

In [ ]:
@tf.function
def augmentation(img):
  img = tf.py_function(hist_eq, [img], tf.float32)
  img = tf.py_function(random_rotation, [img], tf.float32)

  img = tf.reshape(img, (1,96,103,channels))

  return img

# Global Variables

In [ ]:
#Define global variables
channels = 4
image_size = (96,103)
image_shape = image_size + (channels,)
batch_size = 64

rng = tf.random.Generator.from_seed(42, alg='philox')

# Data prep by Aheebwa Steven

In [ ]:
dir =r'C:\Users\HP\Desktop\final_project_recess\Group_F_machine_learning\Recess_Project_Template\resources\datasets\fingerprint_datasets'
classes = ['Altered-Easy', 'Altered-Hard', 'Altered-Medium', 'Real']
classes_map = dict(zip(classes,range(len(classes))))
class_dict = {key:len(glob.glob(f'{dir}/{key}/*.BMP')) for key in classes}

In [ ]:
# Plot class distribution

plt.figure(figsize=(6,6))
sns.barplot(y=list(class_dict.values()), x=list(class_dict.keys()), color='g')
plt.title('Distribution of Classes')
plt.ylabel('Class counts')
plt.xlabel('Classes')
plt.show()

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(dir,
                                                       validation_split=0.2,
                                                       subset="training",
                                                       seed=42,
                                                       image_size=image_size,
                                                       batch_size=batch_size,
                                                       label_mode='categorical',
                                                       labels='inferred'
                                                       )

val_ds = tf.keras.utils.image_dataset_from_directory(dir,
                                                     validation_split=0.2,
                                                     subset="validation",
                                                     seed=42,
                                                     image_size=image_size,
                                                     batch_size=batch_size,
                                                     label_mode='categorical',
                                                     labels='inferred'
                                                     )

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)
print(num_classes)

In [ ]:
def get_class_weights():
  total = sum(class_dict.values())
  class_weights = {}

  for class_name, class_count in class_dict.items():
    class_weights[class_names.index(class_name)] = (1 / class_count) * (total / num_classes)

  return class_weights

In [ ]:
class_weights = get_class_weights()
class_weights

In [ ]:
def visualize(original, augmented):

  def show(ax, image, title):
    ax.set_title(title)
    ax.imshow(image)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

  fig = plt.figure(figsize=(9, 9))

  axs = fig.subplots(1, 2)

  show(axs[0], original, 'original')
  show(axs[1], augmented, 'augmented')

In [ ]:
sample = list(train_ds.take(1))[0][0][:1]
sample_aug = augmentation(sample)

In [ ]:
sample = sample[0].numpy().astype(np.uint8)
sample_aug = sample_aug[0].numpy().astype(np.uint8)

visualize(sample, sample_aug)

In [ ]:
# Joining the augmentated dataset to the original train dataset
train_ds_aug = train_ds.rebatch(1).map(lambda x, y: (augmentation(x), y)).rebatch(batch_size)
train_ds = train_ds.concatenate(train_ds_aug).rebatch(1).shuffle(buffer_size=2048).rebatch(batch_size)

In [ ]:
# Normalising data
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# To optimise the pipeline
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_ds.element_spec